# Music generation with HF's Jukebox model 
For now the PR has not been merged on the official repo yet, and we have to use Arthur's branch `jukebox`. 

In [ ]:
# Install the correct transformer version 
!git clone --branch=jukebox https://github.com/ArthurZucker/transformers.git
%pip install -e ".[dev]"
!sudo apt-get install libsndfile-dev

Lets import a few functionnalities and define the metadatas that will be used


In [ ]:
from transformers import JukeboxModel , JukeboxTokenizer
import time
import torch
from transformers.models.jukebox.modeling_jukebox import load_prompts
metas = dict(
    artist="The weeknd",
    genres="Rap",
    lyrics="""I met a traveller from an antique land,
Who said "Two vast and trunkless legs of stone
Stand in the desert. . . . Near them, on the sand,
Half sunk a shattered visage lies, whose frown,
And wrinkled lip, and sneer of cold command,
Tell that its sculptor well those passions read
Which yet survive, stamped on these lifeless things,
The hand that mocked them, and the heart that fed;
And on the pedestal, these words appear:
My name is Ozymandias, King of Kings;
Look on my Works, ye Mighty, and despair!
Nothing beside remains. Round the decay
Of that colossal Wreck, boundless and bare
The lone and level sands stretch far away
""",
)

In [ ]:
model_id = "ArthurZ/jukebox-1b-lyrics"
model = JukeboxModel.from_pretrained(model_id).eval()
tokenizer = JukeboxTokenizer.from_pretrained(model_id)
tokens = tokenizer(**metas)["input_ids"]

Let's start by generating from scratch only using the conditionning on the meta datas.

In [ ]:
start = time.time()
zs = model.ancestral_sample([i.cuda() for i in tokens], chunk_size=32, sample_length_in_seconds=10, offset = 30)
print("generation time for length : ",time.time()- start)

Now lets load $\frac{40000}{sampling rate}$ seconds of a random audio and generate a continuation! 



In [ ]:
start = time.time()
x = load_prompts(["prompts/test.wav"],40000,model.config)
zs = model.primed_sample(x.cuda(), [i.cuda() for i in tokens], chunk_size=32, sample_length_in_seconds=60)
print("generation time for length : ",time.time()- start)